
4주차 1번째 커널
---
Dynamics of New York City
https://www.kaggle.com/drgilermo/dynamics-of-new-york-city-animation



Import
---



matplotlib.animation\
The easiest way to make a live animation in matplotlib is to use one of the Animation classes.
https://matplotlib.org/3.1.1/api/animation_api.html

matplotlib.cm\
Builtin colormaps, colormap handling utilities, and the ScalarMappable mixin.
https://matplotlib.org/3.1.1/api/cm_api.html

parser = datetime 라이브러리와 같이 쓰여서 일,월,년으로 쉽게 나눠줌\ 
https://dateutil.readthedocs.io/en/stable/

base64 = binary 데이터를 텍스트 형식으로 보이게하거나, ASCII 스트링 포맷으로 나타낼 때 자주 사용\
https://code.tutsplus.com/tutorials/base64-encoding-and-decoding-using-python--cms-25588

파이썬 프로그램 내에서 새로운 프로세스를 만들고 거기에 입출력 파이프를 연결해 리턴코드를 받을 수 있게함. os모듈들을 대체하기 위해 만들어짐.

https://soooprmx.com/archives/5932

In [0]:
from matplotlib import animation # 
from matplotlib import cm

from dateutil import parser

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborClassifier

import base64

from subprocess import check_output
print(check_output(["ls", "../content"]).decode("utf8"))


Read Data
---



In [0]:
df = df[(df.pickup_longitude> xlim[0]) & (df.pickup_longitude < xlim[1])]
# df pick longtitude 열이 lim 위 아래 안에 들어가는 df 만 뽑겠다

In [0]:
plt.plot(longitude, latitude, '.', alpha = 0.4, markersize = 0.05)  # alpha 는 투명도, markersize 는 점의 크기
# '.' 은 플롯할 각 점들의 모양과 색을 말함
# 현재까지 알게 된 것 '.', 'o' = 원, 'bs' = 파랑 사각형 'r--' = 빨간 대쉬, 'g^' = 초록 삼각형
# 이런식으로도 가능(아래)


# evenly sampled time at 200ms intervals
t = np.arange(0., 5., 0.2)
# red dashes, blue squares and green triangles
plt.plot(t, t, 'r--', t, t**2, 'bs', t, t**3, 'g^')
plt.show()


Cluster
---



In [0]:
kemans = KMeans(n_clusters=15, random_state=15, n_init = 10).fit(loc_df)
# kmeans clustering 을 할건데 15개로 나누고 10번의 임의의 seed를 선택해서 그 중에 가장 잘 나눠진 것으로 지정할 것이다
loc_df['label'] = kmeans.labels_ # kmeans 해서 나온 label 들을 array 형태로 가지고 있는듯

loc_df = loc_df.sample(200000) # loc_df 중에서 20만개만 샘플링해라

for label in loc_df.label.unique(): # label 의 중복되지 않은 값의 리스트 중 하나씩 뽑아서
    plt.plot(loc_df.longitude[loc_df.label == label], loc_df.latitude[loc_df.label == label], '.', alpha = 0.3, markersize = 0.3) 
    # df label에 해당하는 경도와 위도 값들을 plotting 해라
    # 이번엔 아까 위보다 더 적은 점들을 플롯할 것 같으니 투명도는 낮췄지만 markersize는 크게 함


In [0]:
# Cluster center 플롯하기 
fig, ax = plt.subplots(figsize = (10, 10))
for label in loc_df.label.unique():
    ax.plot(loc_df.longitude[loc_df.label == label], loc_df.latitude[loc_df.label == label],
            '.', alpha = 0.4, markersize = 0.1, color = 'gray')
    ax.plot(kmeans.cluster_centers_[label, 0], kmeans.cluster_centers_[label, 1], 'o', color = 'r')
    # cluster center 에 대해 label 에 해당하는 경도와 위도 위치에 원의 포맷으로 빨간 색으로 플롯 할 것이다
    ax.annotate(label, (kmeans.cluster_centers_[label, 0], kmeans.cluster_centers_[label, 1]), color = 'b', fontsize = 20)
    # 주석을 달 건데 각 레이블 마다 cluster center 값중 label 에 해당하는 경도와 위도 위치에다가 적을 것이고 색은 블루, 폰트크기는 20으로 적을 것이다
    
    
    ax.set_title('Cluster Centers')
plt.show()



Taxi rides from one cluster to another
---

In [0]:
def animate(hour):
    ax.clear() # 같은 subplot 공간안에 계속 플롯을 해야하므로 clear함수로 지워줘야함. 애초에 다른 공간이면 필요없으나 for 문이나 animation 의 경우 필요
    ax.set_title('Absolute Traffic - Hour' + str(int(hour)) + ': 00')
    plt.figure(figsize = (10, 10))
    for label in loc_df.label.unique():
        ax.plot(loc_df.longitude[loc_df.label == label], loc_df.latitude[loc_df.label == label],
               '.', alpha = 1, markersize = 2, color = 'gray')
        ax.plot(kmeans.cluster_centers_[label,0],kmeans.cluster_centers_[label,1],'o', color = 'r');
    
    # 이중 for 문으로 label과 거기서 도착하는 모든 dest_label 들 탐색
    for label in clusters.label:
        for dest_label in clusters.label:
            num_of_rides = len(df[(df.pickup_cluster == label) & (df.dropoff_cluster == dest_label) & (df.pickup_hour == hour)])
            # 출발지 도착지 pick 시간 모두 맞는 애들의 개수 구하기(나중에 화살 크기 구하기 위해)
            dist_x = clusters.x[clusters.label == label].values[0] - clusters.x[clusters.label == dest_label].values[0] # 출발-도착 경도거리
            dist_y = clusters.y[clusters.label == label].values[0] - clusters.y[clusters.label == dest_label].values[0] # 출발-도착 위도거리
            pct = np.true_divide(num_of_rides, len(df)) # int값으로 때우고 그러지 않는 divide 
            arr = Arrow(clusters.x[clusters.label == label].values, clusters.y[clusters.label == label].values, -dist_x, -dist_y,
                        edgecolor = 'white', width = 15*pct) # Arrow function. 입력으로 label 출발지의 위치를 적고 거기서 -dist 만큼 움직임
            ax.add_patch(arr) # 서브플롯 안에 나타냄
            arr.set_facecolor('g') # 색을 초록색으로

In [0]:
# 전체 24시간 동안의 길이에 비례한 차량의 이동을 나타낸 것이 absolute
# 각 시간당 차량의 이동간의 비율을 고려해서 나타낸 것이 relative


Neighborhood Analysis
---



In [0]:
neighborhood = {-74.0019368351: 'Chelsea',-73.837549761: 'Queens',-73.7854240738: 'JFK',-73.9810421975:'Midtown-North-West',-73.9862336241: 'East Village',
                -73.971273324:'Midtown-North-East',-73.9866739677: 'Brooklyn-parkslope',-73.8690098118: 'LaGuardia',-73.9890572967:'Midtown',-74.0081765545: 'Downtown'
                ,-73.9213024854: 'Queens-Astoria',-73.9470256923: 'Harlem',-73.9555565018: 'Uppe East Side',
               -73.9453487097: 'Brooklyn-Williamsburgt',-73.9745967889:'Upper West Side'}

In [0]:
rides_df = pd.DataFrame(columns = neighborhood.values())
rides_df['name'] = neighborhood.values()
# name 을 index 로 놔두지 않고 굳이 하나의 col 로 빼준 이유는 좀있다 있을 plotly 플로팅에서 필요하기 때문. 

In [0]:
neigh = KNeighborsClassifier(n_neighbors=1) # 가장 가까운 1명의 이웃만 찾는 classifier

neigh.fit(np.array(list(neighborhood.keys())).reshape(-1, 1), list(neighborhood.values()))
# neighborhood 의 values 들은 리스트로 넣고, key들은 리스트로 넣어서 array의 형태로 받게한뒤 각각 label이 되도록 (-1, 1)의 형태로 만들어줌
# 그뒤 fit 시작 여기선 longitude로만 했음. latitude 까지 할수는 없었나?


In [0]:
df['pickup_neighborhood'] = neigh.predict(df.pickup_longitude.values.reshape(-1, 1))
df['dropoff_neighborhood'] = neigh.predict(df.dropoff_longitude.values.reshape(-1, 1))
# 출발지의 가장 가까운 이웃, 도착지의 가장 가까운 이웃 예측

for col in rides_df.columns[:-1]:
    rides_df[col] = rides_df.name.apply(lambda x: len(df[(df.pickup_neighborhood == x) & (df.dropoff_neighborhood == col)]))
# 각 지역에 따라 가장 가까운 이웃들에 간 빈도수 만큼 값을 집어넣어줌
    

heatmap plotly로 그리기. plotly 의 경우 colab에서 사용하기 위해선 1주 2번째 커널에서 썼던 함수가 필요함

In [0]:
def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"</script>
    <script>
    requirejs.config({
    paths: {
    base: '/static/base',
    plotly: 'https:..cdn.plot.ly/plotly-1.5.1.min.js?noext',
    }
    });
    </script>
    '''))

In [0]:
import plotly.offline as py
import plotly.graph_objs as go


configure_plotly_browser_state() # 위 함수 적용
py.init_notebook_mode(connected=True) # 없어도 되는 것 같음

trace = go.Heatmap(z = np.array(rides_df.as_matrix()), # 각 값들을 matrix 형태로 받음
                  x = rides_df.columns[:-1],
                  y = rides_df.columns)
# graph object 를 만들 건데 x축, y 축, z 축의 값은 이러이러 하다. 

layout = dict(title = '<b>Neighbors Interaction<b>',
             titlefont = dict(size = 30,
                             color = ('rgb(100, 100, 100)')),
             margin = dict(t=100,r=100,b=100,l=150),
             yaxis = dict(title = ' <b> From </b>'),
             xaxis = dict(title = '<b> To </b>'))

data = [trace] # object를 리스트로 감싸줌. 왜지?
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'labelled-heatmap') 

out bound inbound 움직임 그림 그리기

In [0]:

ax.plot([0, 250000], [0, 250000], color='r', linewidth = 1)
# 점을 plot해서 선을 그릴건 여기서부터 여기까지의 점을 가지고 빨간색으로 line 두께는 1로 진행해라. 

ax.grid('off')
# 그리드가 나오지 않도록
ax.set_xlim([0, 250000])
ax.set_ylim([0, 250000])
# 두개이상의 axes 를 겹쳐서 plot 하므로 축마다 같은 limit 을 주기 위해서 


Winter vs. Summer
---



In [0]:
df['pickup_month'] = df.pickup_datetime.apply(lambda x: parser.parse(x).month)
# 달에 따라서 date time 을 나눌수 있도록 parser 실행


In [0]:


for col in rides_df.columns[:-1]:
    rides_df[col] = rides_df.name.apply(lambda x: len(df[(df.pickup_neighborhood == x) & (df.dropoff_neighborhood == col) & (df.pickup_month == 6)]))
# neighborhood가 같은 애들, 그리고 pickup month 가 6월인 애들만 계산하기
# 나머지는 위와 동일
